# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weatherpy_data = pd.read_csv("../WeatherPy/cities.csv")
weatherpy_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Goba,35,ET,1603173420,64,7.02,39.98,55.92,4.52
1,Coahuayana Viejo,82,MX,1603173420,89,18.73,-103.68,78.17,1.99
2,Hasaki,20,JP,1603173015,60,35.73,140.83,72.00,11.41
3,Khatanga,84,RU,1603172952,96,71.97,102.50,12.33,6.78
4,Tasiilaq,90,GL,1603172960,100,65.61,-37.64,28.40,6.93
...,...,...,...,...,...,...,...,...,...
549,Bolaños de Calatrava,100,ES,1603173531,40,38.91,-3.66,68.00,10.29
550,Kirensk,99,RU,1603173531,91,57.79,108.11,29.39,12.21
551,Vanavara,100,RU,1603173207,92,60.34,102.28,27.39,12.82
552,Forestville,1,US,1603173531,93,38.85,-76.88,61.00,3.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configuring gmaps
gmaps.configure(api_key=g_key)

#Store lat and long in locations
locations = weatherpy_data[["Lat", "Lng"]]

#Store Humidity in humidity list
humidity = weatherpy_data["Humidity"]

In [10]:
#Creating the heatmap
#use https://developers.google.com/maps/documentation/javascript/heatmaplayer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level =2)
intensity=np.max(humidity)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

#add layer on top of heat_layer 
fig.add_layer(heat_layer)

#Display above figure
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
#Checking intensity - cannot run with fig code above
intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
weather_conditions_df = weatherpy_data.loc[(weatherpy_data["Wind Speed"] <= 10) & (weatherpy_data["Cloudiness"] == 0) & \
                                   (weatherpy_data["Max Temp"] >= 70) & (weatherpy_data["Max Temp"] <= 80)].dropna()

weather_conditions_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
22,Bilma,0,NE,1603173043,16,18.69,12.92,79.97,3.67
63,Assiut,0,EG,1603173433,60,27.18,31.18,73.40,5.82
134,Mvuma,0,ZW,1603173448,27,-19.28,30.53,73.20,5.75
169,Presidencia Roque Sáenz Peña,0,AR,1603173456,51,-26.79,-60.44,78.26,7.16
179,Dalbandin,0,PK,1603173457,17,28.89,64.41,72.61,7.23
182,Masvingo,0,ZW,1603173458,26,-20.06,30.83,76.69,5.06
213,Polokwane,0,ZA,1603173464,53,-23.90,29.47,71.60,2.24
221,Tacuatí,0,PY,1603173466,64,-23.45,-56.58,75.56,2.95
318,Tessalit,0,ML,1603172956,18,20.20,1.01,76.35,7.49
412,Gumdag,0,TM,1603173055,16,39.21,54.59,75.65,2.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#hotel dataframe
hotel_df = weather_conditions_df.loc[:,["City","Country", "Lat", "Lng"]]

#add Hotel Name into column into the DF
hotel_df["Hotel Name"] = ""

#Testing
hotel_df

,City,Country,Lat,Lng,Hotel Name
22,Bilma,NE,18.69,12.92,
63,Assiut,EG,27.18,31.18,
134,Mvuma,ZW,-19.28,30.53,
169,Presidencia Roque Sáenz Peña,AR,-26.79,-60.44,
179,Dalbandin,PK,28.89,64.41,
182,Masvingo,ZW,-20.06,30.83,
213,Polokwane,ZA,-23.90,29.47,
221,Tacuatí,PY,-23.45,-56.58,
318,Tessalit,ML,20.20,1.01,
412,Gumdag,TM,39.21,54.59,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
